# K-Nearest Neighbors (KNN) - No-Framework Implementation

Multi-class classification on the **Covertype (Forest Cover Type)** dataset using pure NumPy.

**Dataset**: 581,012 samples, 54 features, 7 forest cover types  
**Task**: Predict forest cover type from cartographic variables  
**Key Concept**: KNN is a "lazy learner" - no training phase, expensive at prediction time

## What We'll Build From Scratch
- **Euclidean/Manhattan distance** calculation (vectorized)
- **K-nearest neighbor search** using argsort
- **Majority voting** for multi-class prediction
- **Batched prediction** to handle 464K training samples without OOM
- **Distance-weighted voting** (optional advanced feature)


In [1]:
# Standard libraries
import numpy as np
import sys

# Add utils to path
sys.path.append('../..')
from utils.data_loader import load_processed_data
from utils.metrics import accuracy, macro_f1_score, confusion_matrix_multiclass
from utils.visualization import (
    plot_confusion_matrix_multiclass,
    plot_validation_curve,
    plot_per_class_f1
)
from utils.performance import track_performance

print("Imports complete!")

Imports complete!
